In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_instances = pd.read_csv("../data_forecasting/Train.csv")
train_instances.tail()

In [ ]:
train_instances['timestamp'] = train_instances['year'].astype('str') + "-" + train_instances['month'].astype('str')
train_instances.drop(columns={'region', 
                      'district', 
                      'stock_initial', 
                      'stock_received', 
                      'stock_adjustment',
                      'stock_end',
                      'average_monthly_consumption',
                      'stock_stockout_days',
                      'stock_ordered'},
                      inplace=True)

In [ ]:
train_instances['timestamp']=pd.to_datetime(train_instances['timestamp'])
train_instances.index= train_instances.timestamp
train_instances.drop(columns={'timestamp'},inplace=True)

## Sort Dataframe by Date - Causality in Preds
train_instances.sort_values(by='timestamp', inplace=True)
train_instances.tail()

In [ ]:
train_instances['stock_distributed'][train_instances.product_code=='AS27137'].plot()

In [ ]:
train_instances['stock_distributed'][train_instances.product_code=='AS27137']

In [ ]:
all_sites = train_instances.site_code.unique()
len(all_sites)
all_sites

In [ ]:
all_products =list(train_instances.product_code.unique())
print(len(all_products))
all_products

## Dataset Preparation

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
site_encoder = LabelEncoder()
product_encoder = LabelEncoder()
#train_instances.drop(columns=['timestamp'], inplace=True)
train_instances['site_code_encoded'] = site_encoder.fit_transform(train_instances['site_code'])
train_instances['product_code_encoded'] = product_encoder.fit_transform(train_instances['product_code'])
train_instances.tail()

In [ ]:
train_instances['site_code_encoded'].unique()

In [ ]:
train_instances['product_code_encoded'].unique()

In [ ]:
train_instances['stock_distributed'][train_instances.product_code=='AS27137']

In [ ]:
plt.figure(figsize=(20,6))
train_instances['stock_distributed'][train_instances.product_code=='AS27137'].plot()

In [ ]:
product_encoder.transform(['AS27137'])

In [ ]:
plt.figure(figsize=(20,6))
train_instances['stock_distributed'][train_instances.product_code_encoded==6].plot()

In [ ]:
final_dataset=train_instances
#final_dataset=train_instances[['site_code_encoded', 'product_code_encoded', 'stock_distributed']]
final_dataset.tail()

## StatsModels

In [ ]:
import numpy as np
import statsmodels.api as sm

In [ ]:
final_dataset.tail()

In [ ]:
train_size = 0.8
train = final_dataset[:int(train_size*(len(final_dataset)))]
valid = final_dataset[int(train_size*(len(final_dataset))):]

In [ ]:
train.tail()

In [ ]:
valid.tail()

In [ ]:
train[['site_code_encoded','product_code_encoded']]

In [ ]:
model = sm.tsa.statespace.SARIMAX(endog=train['stock_distributed'],
                                  exog=train[['site_code_encoded','product_code_encoded']],
                                  order=(2,0,0),
                                  #order=(1,1,0),
                                  #order=(7,1,7),
                                  enforce_stationarity=True)
sarima = model.fit()
print(sarima.summary())

In [ ]:
preds = sarima.forecast(steps=len(valid), exog=valid[['site_code_encoded','product_code_encoded']])

In [ ]:
preds=preds.reset_index(drop=True)
preds.index=valid.index
preds

In [ ]:
full_preds = pd.concat([valid, preds], axis=1)
full_preds

In [ ]:
tes = full_preds['stock_distributed'][full_preds.product_code_encoded==6]
tes

In [ ]:
pred=full_preds['predicted_mean'][full_preds.product_code_encoded==6]
pre

In [ ]:
prod_label = 7
pd.DataFrame({'test':full_preds['stock_distributed'][full_preds.product_code_encoded==prod_label],
              'pred':full_preds['predicted_mean'][full_preds.product_code_encoded==prod_label]}).plot();
plt.show()

### Filtering by Product

In [ ]:
product_label=6
train_filtered = train[train.product_code_encoded==product_label]
valid_filtered = valid[valid.product_code_encoded==product_label]
train_filtered

In [ ]:
plt.figure(figsize=(20,6))
ax = plt.subplot(111)
ax.bar(train_filtered.index, train_filtered['stock_distributed'], width=10)
ax.xaxis_date()

In [ ]:
model_2 = sm.tsa.statespace.SARIMAX(endog=train_filtered['stock_distributed'],
                                  exog=train_filtered[['site_code_encoded']],
                                  order=(1,0,0))
sarima_2 = model_2.fit()
print(sarima_2.summary())

In [ ]:
valid_filtered[['site_code_encoded']]

In [ ]:
preds_filtered = sarima_2.forecast(steps=len(valid_filtered), exog=valid_filtered[['site_code_encoded']])
preds_filtered

In [ ]:
preds_filtered=preds_filtered.reset_index(drop=True)
preds_filtered.index=valid_filtered.index
full_preds_filtered = pd.concat([valid_filtered, preds_filtered], axis=1)
full_preds_filtered

In [ ]:
plt.figure(figsize=(20,6))
ax = plt.subplot(111)
ax.bar(full_preds_filtered.index, full_preds_filtered['stock_distributed'], width=10)
ax.bar(full_preds_filtered.index, full_preds_filtered['predicted_mean'], width=10)
ax.xaxis_date()

## One Hot Encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def onehot_encode(df, onehot_columns):
    ct = ColumnTransformer(
        [('onehot', OneHotEncoder(drop='first'), onehot_columns)],
        remainder='passthrough'
        )
    return ct.fit_transform(df)

In [ ]:
train.reset_index(drop=True, inplace=True)
train.tail()

In [ ]:
def onehot_encode_pd(df, col_name):
    dummies = pd.get_dummies(df[col_name], prefix=col_name)
    return pd.concat([df, dummies], axis=1).drop(columns=[col_name])

In [ ]:
dummies = pd.get_dummies(train['site_code'], prefix='site_code')
dummies

In [ ]:
new_train = pd.concat([train, dummies], axis=1)
new_train.tail()

## Other Dataset Transformation

In [ ]:
final_df = final_dataset[['year','month','site_code_encoded','product_code_encoded','stock_distributed']].reset_index(drop=True)
final_df

In [ ]:
train_size = 0.8
train = final_df[:int(train_size*(len(final_df)))]
valid = final_df[int(train_size*(len(final_df))):]

In [ ]:
valid.tail()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train_arr = scaler.fit_transform(train[['year','month','site_code_encoded','product_code_encoded']])
X_val_arr = scaler.transform(valid[['year','month','site_code_encoded','product_code_encoded']])

y_train_arr = scaler.fit_transform(train[['stock_distributed']])
y_val_arr = scaler.transform(valid[['stock_distributed']])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from torch.utils.data import TensorDataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

In [ ]:
## Data loaders
batch_size = 64

train_features = torch.Tensor(X_train_arr)
train_targets = torch.Tensor(y_train_arr)
val_features = torch.Tensor(X_val_arr)
val_targets = torch.Tensor(y_val_arr)

train = TensorDataset(train_features, train_targets)
val = TensorDataset(val_features, val_targets)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, h0 = self.rnn(x, h0.detach())
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
class LSTMPredictor(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
        super(LSTMPredictor, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers,
          dropout=0.5
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)
        self.reset_hidden_state()
        
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
        )
    def forward(self, sequences):
        lstm_out, self.hidden = self.lstm(
          sequences.view(len(sequences), self.seq_len, -1),
          self.hidden
        )
        last_time_step = \
          lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred

In [ ]:
X_train_arr.shape[1]

In [ ]:
input_dim = X_train_arr.shape[1]
input_dim

In [ ]:
input_dim = X_train_arr.shape[1]
output_dim = 1
hidden_dim = 64
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-6

In [ ]:
# model_params = {'input_dim': input_dim,
#                 'hidden_dim' : hidden_dim,
#                 'layer_dim' : layer_dim,
#                 'output_dim' : output_dim,
#                 'dropout_prob' : dropout}

model_params = {
    'n_features' : input_dim,
    'n_hidden': hidden_dim,
    'seq_len': output_dim,
    'n_layers': layer_dim
}
model_params

In [ ]:
model = LSTMPredictor(**model_params)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
model

In [ ]:
train_losses = []
val_losses = []

In [ ]:
x, y = next(iter(train_loader))

In [ ]:
model(x)

In [ ]:
num_epocs = 2
model.train()

In [ ]:
optimizer.zero_grad()

In [ ]:
from sklearn.metrics import mean_squared_error

In [1]:
for epoch in range(n_epochs):
    print(f'Epoch {epoch}')
    for step, (x,y) in enumerate(train_loader):
        model.train()
        model.reset_hidden_state()
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, x)
        loss.backward()
        optimizer.step()
        if step%100 == 0:
            for step_valid, (x_valid,y_valid) in val_loader:
                with torch.no_grad():
                    output_valid = model.forward(x_valid)
                mse = mean_squared_error(y_valid, output_valid)
        print(f"Iteration: {step}; Loss: {loss.item()}; MSE: {mse}")

NameError: name 'n_epochs' is not defined

In [ ]:
for t in range(num_epochs):
    for x,y in 
    model.reset_hidden_state()
    y_pred = model(X_train)
    loss = loss_fn(y_pred.float(), y_train)
    if test_data is not None:
      with torch.no_grad():
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred.float(), y_test)
      test_hist[t] = test_loss.item()
      if t % 10 == 0:
        print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
    elif t % 10 == 0:
      print(f'Epoch {t} train loss: {loss.item()}')
    train_hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

In [ ]:
def train_model(
      model,
      train_data,
      train_labels,
      test_data=None,
      test_labels=None
):
  loss_fn = torch.nn.MSELoss(reduction='sum')
  optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
  num_epochs = 60
  train_hist = np.zeros(num_epochs)
  test_hist = np.zeros(num_epochs)
  for t in range(num_epochs):
    model.reset_hidden_state()
    y_pred = model(X_train)
    loss = loss_fn(y_pred.float(), y_train)
    if test_data is not None:
      with torch.no_grad():
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred.float(), y_test)
      test_hist[t] = test_loss.item()
      if t % 10 == 0:
        print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
    elif t % 10 == 0:
      print(f'Epoch {t} train loss: {loss.item()}')
    train_hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  return model.eval(), train_hist, test_hist

## Metrics

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(full_preds['stock_distributed'], full_preds['predicted_mean'])

In [ ]:
mean_squared_error

## Sources

https://www.kaggle.com/poiupoiu/how-to-use-sarimax